**Grader - Please note the starter notebook for this activity has been changed by the instructor for this class. 

In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets

In [3]:
import pandas as pd
df = pd.read_csv("Data/exoplanet_data.csv")
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Read the CSV and Perform Basic Data Cleaning

In [4]:
# Exploratory data analysis
list(df.columns)

['koi_disposition',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_fpflag_co',
 'koi_fpflag_ec',
 'koi_period',
 'koi_period_err1',
 'koi_period_err2',
 'koi_time0bk',
 'koi_time0bk_err1',
 'koi_time0bk_err2',
 'koi_impact',
 'koi_impact_err1',
 'koi_impact_err2',
 'koi_duration',
 'koi_duration_err1',
 'koi_duration_err2',
 'koi_depth',
 'koi_depth_err1',
 'koi_depth_err2',
 'koi_prad',
 'koi_prad_err1',
 'koi_prad_err2',
 'koi_teq',
 'koi_insol',
 'koi_insol_err1',
 'koi_insol_err2',
 'koi_model_snr',
 'koi_tce_plnt_num',
 'koi_steff',
 'koi_steff_err1',
 'koi_steff_err2',
 'koi_slogg',
 'koi_slogg_err1',
 'koi_slogg_err2',
 'koi_srad',
 'koi_srad_err1',
 'koi_srad_err2',
 'ra',
 'dec',
 'koi_kepmag']

In [5]:
# Inspect rows and columns with missing data and decide what to do with them
df = df[df.columns.drop(list(df.filter(regex='err')))]

df.head()


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,...,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


In [6]:
# Drop rows/columns with missing data or you can replace missing data
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   koi_disposition   6991 non-null   object 
 1   koi_fpflag_nt     6991 non-null   int64  
 2   koi_fpflag_ss     6991 non-null   int64  
 3   koi_fpflag_co     6991 non-null   int64  
 4   koi_fpflag_ec     6991 non-null   int64  
 5   koi_period        6991 non-null   float64
 6   koi_time0bk       6991 non-null   float64
 7   koi_impact        6991 non-null   float64
 8   koi_duration      6991 non-null   float64
 9   koi_depth         6991 non-null   float64
 10  koi_prad          6991 non-null   float64
 11  koi_teq           6991 non-null   int64  
 12  koi_insol         6991 non-null   float64
 13  koi_model_snr     6991 non-null   float64
 14  koi_tce_plnt_num  6991 non-null   int64  
 15  koi_steff         6991 non-null   int64  
 16  koi_slogg         6991 non-null   float64


# Select your features (columns)

In [7]:
# Set your y to be koi-disposition and your X to be all other columns
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])



In [8]:
# Drop *unimportant, irrelevant, or duplicate columns
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X, y)

sorted(zip(rf.feature_importances_, X.columns), reverse=True)

[(0.1286575758012663, 'koi_fpflag_co'),
 (0.12721763797696445, 'koi_model_snr'),
 (0.126701543227504, 'koi_fpflag_nt'),
 (0.10335986706085779, 'koi_fpflag_ss'),
 (0.08481713931144624, 'koi_prad'),
 (0.049354572048479664, 'koi_depth'),
 (0.04560783712356872, 'koi_fpflag_ec'),
 (0.041549850697003565, 'koi_period'),
 (0.04024632296190981, 'koi_impact'),
 (0.03330608216817548, 'koi_insol'),
 (0.03096009664303546, 'koi_teq'),
 (0.03063688141451668, 'koi_duration'),
 (0.02640485429136443, 'koi_time0bk'),
 (0.021911332720907062, 'ra'),
 (0.02086101440063755, 'koi_steff'),
 (0.02029863564623416, 'koi_kepmag'),
 (0.02008636380686993, 'koi_srad'),
 (0.020079124809859048, 'dec'),
 (0.019985937629908492, 'koi_slogg'),
 (0.007957330259491163, 'koi_tce_plnt_num')]

In [9]:
X = X[["koi_model_snr","koi_fpflag_nt","koi_fpflag_co","koi_fpflag_ss","koi_prad","koi_fpflag_ec","koi_depth","koi_period"]]
X

,koi_model_snr,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ss,koi_prad,koi_fpflag_ec,koi_depth,koi_period
0,25.8,0,0,0,2.83,0,874.8,54.418383
1,76.3,0,0,1,14.60,0,10829.0,19.899140
2,505.6,0,0,1,33.46,0,8079.2,1.736952
3,40.9,0,0,0,2.75,0,603.3,2.525592
4,40.2,0,0,0,2.77,0,686.0,4.134435
...,...,...,...,...,...,...,...,...
6986,8.4,0,0,0,1.11,1,87.7,8.589871
6987,453.3,0,1,1,29.35,0,1579.2,0.527699
6988,10.6,0,0,0,0.72,0,48.5,1.739849
6989,12.3,0,1,0,1.07,0,103.6,0.681402


# Create a Train Test Split


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, random_state=42)



# Pre-processing

In [ ]:
# Scale your data using appropriate scaler
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Train various  models



## Model 1

In [ ]:
from tensorflow.keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical

In [ ]:
# Instantiate model 1 (e.g., logistic model)
from tensorflow.keras.models import Sequential

model = Sequential()




In [ ]:
from tensorflow.keras.layers import Dense
number_inputs = 8
number_hidden_nodes = 10
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [ ]:
number_classes = 3
model.add(Dense(units=number_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=400,
    shuffle=True,
    verbose=2
)

In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Look at model evalauation metrics - there are more options than just score


## Model 2

In [ ]:
# Instantiate model 2 (e.g., random forest model)
from sklearn.svm import SVC 
grid_model = SVC()
grid_model


In [ ]:
# Fit model using GridSearchCV if it makes sense

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50], 'kernel': ('linear', 'rbf')}
grid = GridSearchCV(grid_model, param_grid, verbose=3)

In [ ]:
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test_scaled, y_test))

In [ ]:
# Look at model evalauation metrics - there are more options than just score

## Feel free to test more models

# Find best model and safe

In [ ]:
# Compare metrics and determine best model


In [ ]:
# Save best model - replace "your_model" with your model variable

import joblib
filename = 'exoplanet_model.sav'
joblib.dump(model, filename)

